# A Multi-View Deep Learning Approach for Cross Domain User Modeling in Recommendation Systems

ref: http://liaha.github.io/models/2016/06/21/dssm-on-tensorflow.html

In [12]:
import tensorflow as tf
import numpy as np

In [13]:
DIM_USER = 10000
DIM_ITEM1 = 100000

DIM_L1 = 400
DIM_L2 = 120

NUM_NEG = 50

In [25]:
def variable_summaries(var, name):
    """Attach a lot of summaries to a Tensor."""
    with tf.name_scope('summaries'):
        mean = tf.reduce_mean(var)
        tf.summary.scalar('mean/' + name, mean)
        with tf.name_scope('stddev'):
            stddev = tf.sqrt(tf.reduce_sum(tf.square(var - mean)))
        tf.summary.scalar('sttdev/' + name, stddev)
        tf.summary.scalar('max/' + name, tf.reduce_max(var))
        tf.summary.scalar('min/' + name, tf.reduce_min(var))
        tf.summary.histogram(name, var)

In [18]:
with tf.name_scope('Input'):
    user_batch = tf.sparse_placeholder(tf.float32, 
                                        shape=[None, DIM_USER], 
                                         name='QueryBatch')
    item1_batch = tf.sparse_placeholder(tf.float32, 
                                        shape=[None, DIM_ITEM1], 
                                        name='DocBatch')

In [27]:
with tf.name_scope('Layer_1'):
    l1_par_range = np.sqrt(2 * 3.0 / (DIM_USER + DIM_L1))
    weight_user_l1 = tf.Variable(tf.random_uniform([DIM_USER, DIM_L1], -l1_par_range, l1_par_range))
    bias_user_l1 = tf.Variable(tf.random_uniform([DIM_L1], -l1_par_range, l1_par_range))
    weight_item1_l1 = tf.Variable(tf.random_uniform([DIM_ITEM1, DIM_L1], -l1_par_range, l1_par_range))
    bias_item1_l1 = tf.Variable(tf.random_uniform([DIM_L1], -l1_par_range, l1_par_range))
    variable_summaries(weight_user_l1, 'L1_user_weights')
    variable_summaries(bias_user_l1, 'L1_user_biases')
    variable_summaries(weight_item1_l1, 'L1_item1_weights')
    variable_summaries(bias_item1_l1, 'L1_item1_biases')
    user_l1 = tf.sparse_tensor_dense_matmul(user_batch, weight_user_l1) + bias_user_l1
    item1_l1 = tf.sparse_tensor_dense_matmul(item1_batch, weight_item1_l1) + bias_item1_l1
    user_l1 = tf.nn.relu(user_l1)
    item1_l1 = tf.nn.relu(item1_l1)

In [31]:
with tf.name_scope('Layer_2'):
    l2_par_range = np.sqrt(2 * 3.0 / (DIM_L1 + DIM_L2))
    weight_user_l2 = tf.Variable(tf.random_uniform([DIM_L1, DIM_L2], -l2_par_range, l2_par_range))
    bias_user_l2 = tf.Variable(tf.random_uniform([DIM_L2], -l2_par_range, l2_par_range))
    weight_item1_l2 = tf.Variable(tf.random_uniform([DIM_L1, DIM_L2], -l2_par_range, l2_par_range))
    bias_item1_l2 = tf.Variable(tf.random_uniform([DIM_L2], -l2_par_range, l2_par_range))
    variable_summaries(weight_user_l2, 'L2_user_weights')
    variable_summaries(bias_user_l2, 'L2_user_biases')
    variable_summaries(weight_item1_l2, 'L2_item1_weights')
    variable_summaries(bias_item1_l2, 'L2_item1_biases')
    user_l2 = tf.matmul(user_l1, weight_user_l2) + bias_user_l2
    item1_l2 = tf.matmul(item1_l1, weight_item1_l2) + bias_item1_l2
    user_l2 = tf.nn.relu(user_l2)
    item1_l2 = tf.nn.relu(item1_l2)